In [0]:
# EXPERIMENTS STATES:
has_default = True
has_grid = True

random_search_amount = 10
group_random_search_amount = 10

stop_criterion = 100
first_group_stop_params = {1: {'max_depth': 3, 'min_child_weight': 3},
 2: {'max_depth': 3, 'min_child_weight': 3},
 3: {'max_depth': 3, 'min_child_weight': 3},
 4: {'max_depth': 3, 'min_child_weight': 3},
 5: {'max_depth': 3, 'min_child_weight': 3},
 6: {'max_depth': 3, 'min_child_weight': 3},
 7: {'max_depth': 3, 'min_child_weight': 5},
 8: {'max_depth': 3, 'min_child_weight': 3},
 9: {'max_depth': 3, 'min_child_weight': 3},
 10: {'max_depth': 3, 'min_child_weight': 4}}
first_group_stop_iters = {1: 243,
 2: 211,
 3: 203,
 4: 243,
 5: 159,
 6: 243,
 7: 132,
 8: 184,
 9: 243,
 10: 243}
# bayes_search = 0
# bayes_group = 0

dataset_name = 'house_prices'
#download_dataset_name_id = 'BNG(credit-g)'

In [0]:
#imports:
import pandas as pd
import numpy as np
import math
import time

from sklearn.utils import shuffle

import xgboost

from scipy.stats import randint as sp_randint # int distribution for random search
from scipy.stats import uniform # float distribution for random search
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from matplotlib import pyplot as plt

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from google.colab import drive 
drive.mount('/mntDrive') 
DIR_TO_SAVE_RES = "/mntDrive/My Drive/GrBoost/Практика/test_full/data/{}/".format(dataset_name)
DIR_TO_DOWNLOAD_DATA = "/mntDrive/My Drive/GrBoost/Практика/test_full/data/input/{}/".format(dataset_name)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [0]:
# IMPORT DATA
"""
import os
from sklearn.datasets import fetch_openml

# Fetch a dataset and create a local copy.
if os.path.exists(DIR_TO_DOWNLOAD_DATA+'{}.npz'.format(dataset_name)):
    with np.load(DIR_TO_DOWNLOAD_DATA+'{}.npz'.format(dataset_name), 'r', allow_pickle=True) as data:
        X = data['X']
        y = data['y']
else:
    dataset = fetch_openml(download_dataset_name_id, version=1)
    X, y = dataset.data, dataset.target
    np.savez(DIR_TO_DOWNLOAD_DATA+'{}.npz'.format(dataset_name), X=X, y=y)

dtrain = xgboost.DMatrix(X, label=y)
"""

"\nimport os\nfrom sklearn.datasets import fetch_openml\n\n# Fetch a dataset and create a local copy.\nif os.path.exists(DIR_TO_DOWNLOAD_DATA+'{}.npz'.format(dataset_name)):\n    with np.load(DIR_TO_DOWNLOAD_DATA+'{}.npz'.format(dataset_name), 'r', allow_pickle=True) as data:\n        X = data['X']\n        y = data['y']\nelse:\n    dataset = fetch_openml(download_dataset_name_id, version=1)\n    X, y = dataset.data, dataset.target\n    np.savez(DIR_TO_DOWNLOAD_DATA+'{}.npz'.format(dataset_name), X=X, y=y)\n\ndtrain = xgboost.DMatrix(X, label=y)\n"

In [0]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
dtrain = xgboost.DMatrix(X, label=y)

# FIRST TEST: RANDOM AND GROUP RANDOM

In [0]:
# default
if not has_default:
  clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)
  grid_search = GridSearchCV(clf, param_grid={'max_depth':[3]}, cv=5, scoring="r2")
  start = time.time()
  grid_search.fit(X, y)
  print("DEFAULT run time: ", (time.time()-start)/60)
  default=pd.DataFrame(grid_search.cv_results_)
  default['experiment_name']='default params'
  default.to_csv(DIR_TO_SAVE_RES+"res_default_{}.csv".format(dataset_name),index=False)

  print(grid_search.best_score_)
# DEFAULT run time:  0.026329147815704345
# 0.6726520660378551

In [0]:
# grid:
if not has_grid:
  param_grid = {'max_depth': [2, 10, 18], 'min_child_weight': [2, 10, 18],
              'alpha':[0.25,0.5,0.75], 'lambda':[0.25,0.5,0.75],
              'subsample':[0.625, 0.75, 0.875],'colsample_bytree':[0.625, 0.75, 0.875]} 

  clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)
  grid_search = GridSearchCV(clf, param_grid=param_grid, cv=5, scoring="r2")

  start = time.time()
  grid_search.fit(X, y)
  print("GRID run time: ", (time.time()-start)/60)

  grid_search_df=pd.DataFrame(grid_search.cv_results_)
  grid_search_df['experiment_name']='grid_search'
  grid_search_df.to_csv(DIR_TO_SAVE_RES+"res_grid_search_{}.csv".format(dataset_name),index=False)
  print(grid_search.best_score_)
# GRID run time:  27.905986074606577
# 0.6828739254131614

In [0]:
# random search:
if random_search_amount<10:
  n_iter_search = 729
  param_dist = {"max_depth": sp_randint(1, 20),
                "min_child_weight": sp_randint(1, 20),
                "alpha": uniform(loc=0, scale=1),
                "lambda": uniform(loc=0, scale=1),
                "subsample":uniform(loc=0.5, scale=0.4),
                "colsample_bytree":uniform(loc=0.5, scale=0.4)}

  for i in range(random_search_amount+1, 11):
    clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                        n_iter=n_iter_search, cv=5, scoring="r2")
    start = time.time()
    random_search.fit(X, y) 
    print("RS run time: ", (time.time()-start)/60)
    res_rand=pd.DataFrame(random_search.cv_results_)
    subset = res_rand.loc[:,['param_max_depth', 'param_min_child_weight', 
                            'param_alpha','param_lambda',
                            'param_colsample_bytree', 'param_subsample']].drop_duplicates().index
    res_rand = res_rand.loc[subset]

    while res_rand.shape[0]<n_iter_search:
      random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                        n_iter=(n_iter_search-random_search.shape[0]), cv=5, scoring="r2")
      random_search.fit(X, y) 
      res_rand = pd.concat([res_rand, random_search], sort = False)
      subset = res_rand.loc[:,['param_max_depth', 'param_min_child_weight', 
                            'param_alpha','param_lambda',
                            'param_colsample_bytree', 'param_subsample']].drop_duplicates().index
      res_rand = res_rand.loc[subset]

    res_rand['run_number']=i

    res_rand['experiment_name']="random_search"
    res_rand.to_csv(DIR_TO_SAVE_RES+"res_rand_{}_{}.csv".format(dataset_name, i), index=False)

"""
RS run time:  27.66507022380829
RS run time:  28.061819970607758
RS run time:  27.44816317160924
RS run time:  27.645976877212526
RS run time:  26.519640775521598
RS run time:  27.433769524097443
RS run time:  28.03506429195404
RS run time:  27.815785245100656
"""

'\nRS run time:  27.66507022380829\nRS run time:  28.061819970607758\nRS run time:  27.44816317160924\nRS run time:  27.645976877212526\nRS run time:  26.519640775521598\nRS run time:  27.433769524097443\nRS run time:  28.03506429195404\nRS run time:  27.815785245100656\n'

In [0]:
# random group search:
if group_random_search_amount<10:
  n_iter_search = 243

  param_dist_group_1 = {"max_depth": sp_randint(1, 20),
                    "min_child_weight": sp_randint(1, 20)}

  param_dist_group_2 = {"alpha": uniform(loc=0, scale=1),
                    "lambda": uniform(loc=0, scale=1)}

  param_dist_group_3 = {"subsample":uniform(loc=0.5, scale=0.4),
                    "colsample_bytree":uniform(loc=0.5, scale=0.4)}


  for i in range(group_random_search_amount+1, 7):
    #first group:
    clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_1,
                                          n_iter=n_iter_search, cv=5, scoring="r2")

    start = time.time()
    random_search.fit(X, y)
    print("RS for first group: ",(time.time() - start)/60)


    res_rand_1=pd.DataFrame(random_search.cv_results_)
    res_rand_1['experiment_name']='random search for 1 group'      
    res_rand_1['run_number']=i

    
    subset = res_rand_1.loc[:,['param_max_depth', 'param_min_child_weight']].drop_duplicates().index
    res_rand_1 = res_rand_1.loc[subset]
    print(res_rand_1.shape)
    while res_rand_1.shape[0]<n_iter_search:
        random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_1,
                                          n_iter=(n_iter_search-res_rand_1.shape[0]), cv=5, scoring="r2")
        random_search.fit(X, y) 
        res_rand_1 = pd.concat([res_rand_1, pd.DataFrame(random_search.cv_results_)], sort = False).reset_index(drop=True)
        subset = res_rand_1.loc[:,['param_max_depth', 'param_min_child_weight']].drop_duplicates().index
        res_rand_1 = res_rand_1.loc[subset]
    print(res_rand_1.shape)
    res_rand_1.to_csv(DIR_TO_SAVE_RES+"res_rand_group1_{}_{}.csv".format(dataset_name, i), index=False)
    max_score = res_rand_1['mean_test_score'].max()
    df_max = res_rand_1.loc[res_rand_1['mean_test_score']==max_score]
    clf.set_params(**df_max['params'].values[0])

    #second group:
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_2,
                                          n_iter=n_iter_search, cv=5, scoring="r2")

    start = time.time()
    random_search.fit(X, y)
    print("RS for second group: ",(time.time() - start)/60)

    res_rand_2=pd.DataFrame(random_search.cv_results_)
    res_rand_2['experiment_name']='random search for 2 group' 
    res_rand_2['run_number']=i

    res_rand_2.to_csv(DIR_TO_SAVE_RES+"res_rand_group2_{}_{}.csv".format(dataset_name, i), index=False)
    clf.set_params(**random_search.best_params_)

    #third group:
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_3,
                                          n_iter=n_iter_search, cv=5, scoring="r2")

    start = time.time()
    random_search.fit(X, y)
    print("RS for third group: ",(time.time() - start)/60)

    res_rand_3=pd.DataFrame(random_search.cv_results_)
    res_rand_3['experiment_name']='random search for 3 group' 
    res_rand_3['run_number']=i
    res_rand_3.to_csv(DIR_TO_SAVE_RES+"res_rand_group3_{}_{}.csv".format(dataset_name, i), index=False)



RS for first group:  8.946099249521891
(184, 17)
(243, 17)
RS for second group:  5.284519712130229
RS for third group:  5.361768352985382
RS for first group:  9.198115090529123
(171, 17)
(243, 17)
RS for second group:  5.282854831218719
RS for third group:  5.360952051480611
RS for first group:  8.984747664133709
(182, 17)
(243, 17)
RS for second group:  5.282317062218984
RS for third group:  5.361859313646952
RS for first group:  8.706163032849629
(179, 17)
(243, 17)
RS for second group:  5.28796079158783
RS for third group:  5.366481220722198
RS for first group:  9.23592178026835
(179, 17)
(243, 17)
RS for second group:  5.2916605512301125
RS for third group:  5.37176075776418


KeyboardInterrupt: ignored

# SECOND TEST: STOP CRITERION FOR RANDOM

In [0]:
def group_best_params_with_stop(df, stop_iter_num, left_iter_num, right_iter_num):
    best_params = {}
    stop_iter_nums = {}
    best_score = {}

    tmp_df = df.loc[(df['iter_num'] >= left_iter_num) & (df['iter_num'] <= right_iter_num)].copy() #take group
    for i in tmp_df.run_number.unique(): 
      tmp_df_i = tmp_df.loc[tmp_df.run_number==i] #take run
      #cut iterations:
      tmp_df_i_copy = tmp_df_i.copy()
      tmp_df_i_copy.loc[:, "mean_test_score"] = tmp_df_i.loc[:, "mean_test_score"].cummax()      

      try:
          curr_max=tmp_df_i_copy.groupby("mean_test_score").count().reset_index().sort_values(by = "mean_test_score")
          max_score = curr_max.loc[curr_max["mean_test_score"]>=stop_iter_num].index[0] #get first interval with iter_amount more than stop_iter_num
      except:
          max_score = tmp_df_i_copy.mean_test_score.max()
      
      tmp_stop = tmp_df_i.loc[tmp_df_i["mean_test_score"]==max_score].sort_values(by="iter_num").iloc[0]#get first element with best_score

      best_params[i] = tmp_stop['params']
      #best_score[i] = tmp_stop['mean_test_score']

      last_iter_num = tmp_stop['iter_num'] + stop_iter_num - 1
      if last_iter_num > right_iter_num:
          last_iter_num = right_iter_num

      stop_iter_nums[i] = last_iter_num
    return best_params, stop_iter_nums#, best_score       
   

In [0]:
#for params_set_num in best_params_first_group:
if first_group_stop_params:
  param_dist_group_2 = {"alpha": uniform(loc=0, scale=1),
                  "lambda": uniform(loc=0, scale=1)}

  param_dist_group_3 = {"subsample":uniform(loc=0.5, scale=0.4),
                    "colsample_bytree":uniform(loc=0.5, scale=0.4)}
  for params_set_num in range(1,11):
    clf = xgboost.XGBRegressor(tree_method = "gpu_hist", gpu_id=0, verbosity=0)
    clf.set_params(**first_group_stop_params[params_set_num])
    n_iter_search_sec = int(round((729 - first_group_stop_iters[params_set_num])/2))
    #second group:
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_2,
                                        n_iter=n_iter_search_sec, cv=5, scoring="r2")

    random_search.fit(X, y)

    res_rand_2=pd.DataFrame(random_search.cv_results_)
    res_rand_2['iter_num'] = range(first_group_stop_iters[params_set_num]+1, 
                                  first_group_stop_iters[params_set_num]+n_iter_search_sec+1)
    res_rand_2['experiment_name']='random search for 2 group' 
    res_rand_2['run_number']=params_set_num
    res_rand_2.to_csv(DIR_TO_SAVE_RES+'GR_S_2_group_with_{}_{}_{}.csv'.format(stop_criterion, params_set_num, dataset_name), index=False)
    #fit last suitable params
    best_params_second_group, sec_stop_iter_nums = group_best_params_with_stop(res_rand_2, 
                                                          stop_iter_num = stop_criterion, 
                                                          left_iter_num=first_group_stop_iters[params_set_num]+1, 
                                                          right_iter_num=first_group_stop_iters[params_set_num]+n_iter_search_sec+1)
    clf.set_params(**best_params_second_group[params_set_num])
    
    print('done for second group')

    #third group:
    n_iter_search_third = 729 - sec_stop_iter_nums[params_set_num]
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist_group_3,
                                          n_iter=n_iter_search_third, cv=5, scoring="r2")
    random_search.fit(X, y)

    res_rand_3=pd.DataFrame(random_search.cv_results_)
    res_rand_3['iter_num'] = range(sec_stop_iter_nums[params_set_num]+1, 729+1)
    res_rand_3['experiment_name']='random search for 3 group' 
    res_rand_3['run_number']=params_set_num

    best_params_third_group, third_stop_iter_nums = group_best_params_with_stop(res_rand_3, 
                                                          stop_iter_num = stop_criterion, 
                                                          left_iter_num=sec_stop_iter_nums[params_set_num]+1, 
                                                          right_iter_num=729)
    res_rand_3=res_rand_3.loc[(res_rand_3['run_number']==params_set_num) & \
                            (res_rand_3['iter_num']<=int(third_stop_iter_nums[params_set_num]))]

    res=pd.concat([res_rand_2,res_rand_3], sort=False)

    sec_idxs = res.loc[res['experiment_name']=='random search for 2 group','iter_num'].values
    third_idxs = res.loc[res['experiment_name']=='random search for 3 group','iter_num'].values
    intersection = np.intersect1d(third_idxs, sec_idxs)

    res = res.loc[~((res['experiment_name']=='random search for 2 group') & (res['iter_num'].isin(intersection)))]

    res.to_csv(DIR_TO_SAVE_RES+'GR_S_2_and_3_groups_with_{}_{}_{}.csv'.format(stop_criterion, params_set_num, dataset_name), index=False)
    print(first_group_stop_iters[params_set_num]+res.shape[0])
    print('done for third group')

done for second group
661
done for third group


# THIRD TEST: BAYES AND GROUPED BAYES

# FOURTH TEST: STOP CRITERION FOR BAYES